## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd

import requests

import gmaps

import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Longyearbyen,SJ,78.2186,15.6401,49.84,87,75,3.44,broken clouds
1,1,Ambon,ID,-3.6954,128.1814,76.23,16,20,8.05,few clouds
2,2,Cherskiy,RU,68.7500,161.3000,47.28,45,99,11.86,overcast clouds
3,3,Dikson,RU,73.5069,80.5464,43.18,93,100,15.77,light rain
4,4,Beyneu,KZ,45.3167,55.2000,86.27,33,91,16.17,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
minimum_temp = float(input("What is the minimun temperature you would like for your trip? "))
maximum_temp = float(input("What is the maximun temperature you would like for your trip? "))

In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= minimum_temp) & (city_data_df["Max Temp"] <= maximum_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Ambon,ID,-3.6954,128.1814,76.23,16,20,8.05,few clouds
4,4,Beyneu,KZ,45.3167,55.2000,86.27,33,91,16.17,overcast clouds
5,5,Victoria,HK,22.2855,114.1577,85.32,82,0,3.00,clear sky
6,6,Roma,US,43.2128,-75.4557,83.88,67,75,10.36,broken clouds
10,10,Kapaa,US,22.0752,-159.3190,84.52,70,40,16.11,scattered clouds


In [11]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

no_empty_df = preferred_cities_df.dropna(axis="index", how="any")

print(len(no_empty_df))

no_empty_df.head()

384


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Ambon,ID,-3.6954,128.1814,76.23,16,20,8.05,few clouds
4,4,Beyneu,KZ,45.3167,55.2000,86.27,33,91,16.17,overcast clouds
5,5,Victoria,HK,22.2855,114.1577,85.32,82,0,3.00,clear sky
6,6,Roma,US,43.2128,-75.4557,83.88,67,75,10.36,broken clouds
10,10,Kapaa,US,22.0752,-159.3190,84.52,70,40,16.11,scattered clouds


In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = no_empty_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"

hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Ambon,ID,76.23,few clouds,-3.6954,128.1814,
4,Beyneu,KZ,86.27,overcast clouds,45.3167,55.2000,
5,Victoria,HK,85.32,clear sky,22.2855,114.1577,
6,Roma,US,83.88,broken clouds,43.2128,-75.4557,
10,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,
11,Mahebourg,MU,69.26,scattered clouds,-20.4081,57.7000,
12,Hobyo,SO,75.47,few clouds,5.3505,48.5268,
13,Aswan,EG,87.15,clear sky,24.0934,32.9070,
15,Ternate,ID,80.65,overcast clouds,0.8000,127.4000,
17,Sur,OM,86.41,overcast clouds,22.5667,59.5289,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 

    hotel = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]

    except (IndexError):
        print("Hotel not found...Skipping the City.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        

Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
Hotel not found...Skipping the City.
H

In [15]:
# 7. Drop the rows where there is no Hotel Name.

hotel_clean_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]

hotel_clean_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Ambon,ID,76.23,few clouds,-3.6954,128.1814,Swiss-Belhotel Ambon
4,Beyneu,KZ,86.27,overcast clouds,45.3167,55.2000,NUR
5,Victoria,HK,85.32,clear sky,22.2855,114.1577,Mini Central
6,Roma,US,83.88,broken clouds,43.2128,-75.4557,Wingate By Wyndham Rome
10,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Mahebourg,MU,69.26,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
12,Hobyo,SO,75.47,few clouds,5.3505,48.5268,Iftin hotel hobyo
13,Aswan,EG,87.15,clear sky,24.0934,32.9070,Nuba Nile
15,Ternate,ID,80.65,overcast clouds,0.8000,127.4000,Sahid Bela Ternate Hotel
17,Sur,OM,86.41,overcast clouds,22.5667,59.5289,Sur Plaza Hotel


In [16]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>

<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>

<dt><b>City</b></dt><dd>{City}</dd>

<dt><b>Country</b></dt><dd>{Country}</dd>

<dt><b>Current Weather</b></dt><dd>{Weather Description}</dd>

<dt><b>Max Temp</b></dt><dd>{Max Temp} °F</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))